In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
import pandas as pd

path = '/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/data_v0_embeddings.pkl'

data = pd.read_pickle(path)
data

,_id,pub_date,chosen_type,text,what,when,where,who,why,how,coords,embedding
0,nyt://article/462daa07-2b8b-5833-afb5-209a968a...,2015-07-01 00:03:44,lead_paragraph,LONDON — Rossini’s opera “Guillaume Tell” has ...,is forced to shoot an arrow,None,LONDON,the hero,Rossini ’s opera “ Guillaume Tell ”,has long been known for the scene in which the,"(51.5073219, -0.1276474)","[0.06282554, -0.037304644, -0.024440732, -0.01..."
1,nyt://article/0a038481-b145-57b3-a94b-701f0e9b...,2015-07-01 00:05:47,lead_paragraph,PARIS — Sometimes a metaphor lands at your fee...,None,last Friday,PARIS,None,This,"— Sometimes a metaphor lands at your feet , li...","(48.8588897, 2.3200410217200766)","[-0.06251681, -0.031227265, -0.0527851, -0.051..."
2,nyt://slideshow/3f39a127-f709-5e2e-8c94-2864fe...,2015-07-01 00:07:39,abstract,"Images of calcio storico in Florence, Italy, a...",None,None,"Florence , Italy",None,None,a centuries-old competition with few rules and...,"(43.7698712, 11.2555757)","[0.06494084, -0.025435511, -0.016922276, -0.01..."
3,nyt://article/1565acdd-a256-5f81-93df-232b1acb...,2015-07-01 00:09:33,lead_paragraph,"Theodore Weesner, a novelist who mined his way...",was 79,Thursday,"Portsmouth , N.H",novelist,None,his wayward youth for the stuff of his celebra...,"(43.0702223, -70.7548621)","[-0.032611, 0.0026771696, -0.0074188053, -0.07..."
4,nyt://article/694f7d7c-304a-5370-aff0-731aa012...,2015-07-01 00:19:40,lead_paragraph,"DEIR AL-BALAH, Gaza Strip — One bomb hit a Ham...",exploded in a trash can,A few days later,Gaza Strip,another,a small one,in northern Gaza .,"(31.4432234, 34.360007)","[-0.016562575, 0.016165249, 0.081286825, -0.05..."
...,...,...,...,...,...,...,...,...,...,...,...,...
157745,nyt://article/51f01df3-5548-5f97-8d94-ea64d449...,2021-07-31 19:28:13,lead_paragraph,"PROVINCETOWN, Mass. — By the Fourth of July, P...",None,the Fourth of July,"PROVINCETOWN , Mass.",None,Restaurants,a prepandemic thrum .,"(42.0525612, -70.1858639)","[-0.035350244, 0.04796858, 0.0746137, -0.03549..."
157746,nyt://video/9afc7520-16a5-5cf8-8d37-e4d4f7d9fd05,2021-07-31 19:34:53,abstract,"Dr. Tedros Adhanom Ghebreyesus, the World Heal...",None,None,Africa,None,"Dr. Tedros Adhanom Ghebreyesus , the World Hea...",a worldwide rise in Covid-19 cases has dispara...,"(11.5024338, 17.7578122)","[0.054478098, -0.021061463, 0.0074481536, -0.0..."
157747,nyt://article/5300b238-e7cc-5d4e-bda0-59c450b1...,2021-07-31 21:13:08,lead_paragraph,At least 233 staff members at two major San Fr...,"vaccinated , tested positive for the coronavirus",this month,San Francisco,At least 233 staff members at two major San Fr...,None,At least 233 staff members at two major San Fr...,"(37.7790262, -122.419906)","[-0.0048520393, -0.01510062, -0.02040333, 0.01..."
157748,nyt://article/8616f3bd-e981-5c1b-8532-146f126b...,2021-07-31 21:34:38,lead_paragraph,"PARIS — Pascale Collino, 64, is far more afrai...",None,None,PARIS,None,she,is far more afraid of the Covid-19 vaccines th...,"(48.8588897, 2.3200410217200766)","[0.016033476, -0.019411247, -0.047736764, -0.0..."


In [62]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [63]:
def remove_stopwords(text,stop_words):
  
  # tudo para caixa baixa
  s = str(text).lower() 

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

In [64]:
def stemming(tokens,stemmer):
  tokens_stems = [stemmer.stem(word) for word in tokens]
  return tokens_stems

In [65]:
def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('english'), stemmer=PorterStemmer()):
  tokens = remove_stopwords(doc,stop_words)
  return stemming(tokens,stemmer)

In [66]:
before = data.isna().sum()

In [67]:
from sklearn import neighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
import numpy as np

components = ['what', 'who', 'why', 'how'] #retirada a componente when pois possui apenas datas relativas e a componente where
components_indexes = [4, 7, 8, 9]

threshold = 0.3

for component_index, component in zip(components_indexes, components):
    print('Vectorizing component: ' + component)

    corpus = data[component].fillna('').to_list()
    
    vectorizer = CountVectorizer(tokenizer = meu_tokenizador, max_features = 5000)
    X = vectorizer.fit_transform(corpus)
    #X = normalize(X)
    
    nn = NearestNeighbors(n_neighbors = 2, metric = 'cosine', algorithm = 'auto', n_jobs = -1).fit(X)
    distances, indices = nn.kneighbors(X)
    
    filter = distances < threshold
    data.iloc[(~(filter[:, 0] & filter[:,1])).flatten(), component_index] = None

Vectorizing component: what
Vectorizing component: who
Vectorizing component: why
Vectorizing component: how


In [68]:
after = data.isna().sum()

In [69]:
pd.DataFrame({'Antes': before, 'Depois': after})

,Antes,Depois
_id,0,0
pub_date,0,0
chosen_type,0,0
text,0,0
what,70824,105590
when,86097,86097
where,0,0
who,70824,102090
why,52412,110668
how,6839,113326


In [70]:
from sklearn.model_selection import train_test_split
from datetime import datetime

a = 0
month = datetime.strptime(data.iloc[0]['pub_date'], '%Y-%m-%d %H:%M:%S').month

train_df = pd.DataFrame(columns = data.columns)
test_df = pd.DataFrame(columns = data.columns)

for index, row in data.iterrows():
    actual_month = datetime.strptime(row['pub_date'], '%Y-%m-%d %H:%M:%S').month
    if actual_month != month:
       train, test = train_test_split(data.iloc[a:index, :], test_size = 0.3)
       
       train_df = train_df.append(train)
       test_df = test_df.append(test)

       month = actual_month
       a = index

train, test = train_test_split(data.iloc[a:, :], test_size = 0.3)

train_df = train_df.append(train).reset_index(drop=True)
test_df = test_df.append(test).reset_index(drop=True)

In [71]:
pickle_save_path = '/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/data_v0_embeddings.pkl'

train_df.to_pickle('/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/train_data_v0_embeddings.pkl')
test_df.to_pickle('/content/drive/Shareddrives/TCC 2022 - Marcelo/lp - paulo/test_data_v0_embeddings.pkl')